In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
import torch
print(torch.__file__)


/usr/local/lib/python3.10/dist-packages/torch/__init__.py


In [3]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
classifier('we are very happy')

[{'label': 'POSITIVE', 'score': 0.9998835325241089}]

In [5]:
classifier('ananth is not happy')

[{'label': 'NEGATIVE', 'score': 0.9997408986091614}]

In [11]:
classifier('my dog is bit sad and more happy')

[{'label': 'POSITIVE', 'score': 0.9985768795013428}]

In [10]:
classifier('The meeting is scheduled for 3 PM tomorrow') disitlbert

[{'label': 'POSITIVE', 'score': 0.9864764213562012}]

In [12]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline with the specific model
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

# Test the classifier
result = classifier("This is a great product!")
print(result)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': '5 stars', 'score': 0.8600963354110718}]


In [15]:
result = classifier("im sad")
print(result)

[{'label': '1 star', 'score': 0.30779406428337097}]


In [19]:
result = classifier("நான் வருத்தமாக இருக்கிறேன்")
print(result)

[{'label': '5 stars', 'score': 0.43927085399627686}]


In [20]:
result = classifier("Je suis triste")
print(result)

[{'label': '1 star', 'score': 0.49396082758903503}]


In [22]:
result = classifier("The product is okay, but it could be improved.")
print(result)

[{'label': '3 stars', 'score': 0.770152747631073}]


In [23]:
from transformers import AutoTokenizer , TFAutoModelForSequenceClassification

In [24]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [25]:
result = classifier("i'm batman")
print(result)

[{'label': '5 stars', 'score': 0.3929806053638458}]


In [26]:
result = classifier("i'm not a  batman")
print(result)

[{'label': '1 star', 'score': 0.3500869870185852}]


In [34]:
import pandas as pd
import re
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

# Function to load and preprocess the WhatsApp chat file
def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    # Define regex for timestamp and group messages
    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        # Clean up each line by removing the timestamp and ignoring group-related messages
        line = line.strip()

        # Skip empty lines and group-related messages
        if not line or re.match(group_message_regex, line):
            continue

        # Remove timestamp
        line = re.sub(timestamp_regex, "", line).strip()

        # Check if the message has a valid format: username: message
        if ": " in line:
            user_message = line.split(": ", 1)
            if len(user_message) == 2:
                user, message = user_message
                messages.append(message)

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=["message"])
    return df

# Load the model and tokenizer for sentiment analysis
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Function to perform sentiment analysis on each message
def analyze_sentiment(df):
    sentiments = []
    for message in df['message']:
        result = classifier(message)
        sentiments.append(result[0]['label'])  # Get the sentiment label
    df['sentiment'] = sentiments
    return df

# Main function to process the chat file and perform sentiment analysis
def main(chat_file_path):
    # Load the WhatsApp chat file into a DataFrame
    df = load_chat(chat_file_path)

    # Perform sentiment analysis
    df_with_sentiments = analyze_sentiment(df)

    # Display the result
    print(df_with_sentiments)

# Run the program with the path to your WhatsApp chat file
if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"  # Provide the path to your WhatsApp chat file
    main(chat_file_path)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


                                      message sentiment
0                                     Hi guys   5 stars
1                                      hi bro   3 stars
2                       Hey! How are you all?   5 stars
3                                  Hello guys   5 stars
4                                  doing good   4 stars
..                                        ...       ...
76                                     France   5 stars
77           argerntina only for my man Messi    1 star
78                                     Great!   5 stars
79  Argentina will win the world cup for sure   5 stars
80                                   Will see   5 stars

[81 rows x 2 columns]
